In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

/Users/shwetasood/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = np.load('./dataset_abundance_ibd.npy')
label = pd.read_csv('./ibdphy_y.csv').iloc[:,1]

label =tf.one_hot(label,depth=2)


In [3]:
data.shape

(110, 443)

In [6]:
label.shape

(110, 2)

In [5]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                        log_device_placement=True))

label = sess.run(label)

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 443])
y_ = tf.placeholder(tf.float32, shape=[None, 2])
W = tf.Variable(tf.truncated_normal(shape=[443,10],stddev=0.1))
#dropped_w = tf.nn.dropout(W,0.8)
dropped_w = W
b = tf.Variable(tf.truncated_normal(shape=[10],stddev=0.1))
a = tf.nn.relu(tf.matmul(x,dropped_w) + b)
W2 = tf.Variable(tf.truncated_normal(shape=[10,2],stddev=0.1))
#dropped_w2 = tf.nn.dropout(W2,0.8)
dropped_w2 = W2
b2 = tf.Variable(tf.truncated_normal(shape=[2],stddev=0.1))
y = tf.matmul(a,dropped_w2) + b2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [10]:
indices = np.random.permutation(101)


xtrain = data[indices[0:77]]
ytrain = label[indices[0:77]]

xtest = data[indices[78:101]]
ytest = label[indices[78:101]]

In [166]:
saved_indices = indices

In [15]:
with tf.device("/gpu:0"):
    sess.run(tf.global_variables_initializer())
    batch_size = 50
    for epoch in range(1,200):
        for i in range(0,162,batch_size):
            if(i+batch_size<len(xtrain)):
                xbatch = xtrain[i:i+batch_size]
                ybatch = ytrain[i:i+batch_size]
            else:
                xbatch = xtrain[i:len(xtrain)]
                ybatch = ytrain[i:len(xtrain)]

            sess.run(train_step,feed_dict={x:xbatch, y_: ybatch})
        print("*******************************************************************************")
        print("Epoch : "+ str(epoch))
        print("Test Accuracy : ")
        print(sess.run(accuracy,feed_dict={x: xtest, y_: ytest}))
        print("Train Accuracy : ")
        print(sess.run(accuracy,feed_dict={x: xtrain, y_: ytrain}))


*******************************************************************************
Epoch : 1
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 2
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 3
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 4
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 5
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 6
Test Accuracy : 
0.65217394
Train Accuracy : 
0.8181818
*******************************************************************************
Epoch : 7
Test Accuracy : 
0.65217394
Train 

0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 83
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 84
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 85
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 86
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 87
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 88
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 89
Test Accuracy : 
0.8695652
Train Accu

0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 144
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 145
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 146
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 147
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 148
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 149
Test Accuracy : 
0.8695652
Train Accuracy : 
1.0
*******************************************************************************
Epoch : 150
Test Accuracy : 
0.8695652
Tra

In [34]:
indices_0 = (np.argmax(ytrain,axis=1)==0)
indices_0 = np.where(indices_0==1)

indices_1 = (np.argmax(ytrain,axis=1)==1)
indices_1= np.where(indices_1==1)

indices_1 = indices_1[0]
indices_0 = indices_0[0]

print(len(indices_1))
print(len(indices_0))

14
63


In [44]:
with tf.device("/gpu:0"):
    sess.run(tf.global_variables_initializer())
    batch_size = 50
    for epoch in range(1,100):
        random_1 = np.random.permutation(len(indices_1))
        random_2 = np.random.permutation(len(indices_0))
        indices_1 = indices_1[random_1]
        indices_0 = indices_0[random_2]
        
        xbatch_1 = xtrain[indices_1[0:7]]
        ybatch_1 = ytrain[indices_1[0:7]]
        xbatch_2 = xtrain[indices_0[0:7]]
        ybatch_2 = ytrain[indices_0[0:7]]
        
        xbatch = np.concatenate((xbatch_1,xbatch_2))
        ybatch  = np.concatenate((ybatch_1,ybatch_2))
        
        final_shuffle = np.random.permutation(14)
        xbatch = xbatch[final_shuffle]
        ybatch = ybatch[final_shuffle]
        sess.run(train_step,feed_dict={x:xbatch, y_: ybatch})
        print("*******************************************************************************")
        print("Epoch : "+ str(epoch))
        print("Test Accuracy : ")
        print(sess.run(accuracy,feed_dict={x: xtest, y_: ytest}))
        print("Train Accuracy : ")
        print(sess.run(accuracy,feed_dict={x: xtrain, y_: ytrain}))

*******************************************************************************
Epoch : 1
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 2
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 3
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 4
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 5
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 6
Test Accuracy : 
0.3478261
Train Accuracy : 
0.18181819
*******************************************************************************
Epoch : 7
Test Accuracy : 
0.3478261
Train A

0.9480519
*******************************************************************************
Epoch : 80
Test Accuracy : 
0.9130435
Train Accuracy : 
0.9480519
*******************************************************************************
Epoch : 81
Test Accuracy : 
0.9130435
Train Accuracy : 
0.90909094
*******************************************************************************
Epoch : 82
Test Accuracy : 
0.9130435
Train Accuracy : 
0.90909094
*******************************************************************************
Epoch : 83
Test Accuracy : 
0.8695652
Train Accuracy : 
0.90909094
*******************************************************************************
Epoch : 84
Test Accuracy : 
0.8695652
Train Accuracy : 
0.90909094
*******************************************************************************
Epoch : 85
Test Accuracy : 
0.9130435
Train Accuracy : 
0.90909094
*******************************************************************************
Epoch : 86
Test Accuracy : 
0

In [40]:
from sklearn.metrics import confusion_matrix

test_y = ytest
Y = sess.run(tf.nn.softmax(y), feed_dict = {x:xtest, y_:ytest})
print(confusion_matrix(np.argmax(test_y, axis = 1), np.argmax(Y, axis = 1)))

[[14  1]
 [ 2  6]]
